In [1]:
import datetime as dt
from utility import (
    read_sql,
    fd_basicinfo,
    fd_typeclass,
    nearest_report_date
)
import pandas as pd

In [2]:
# 获取回溯的报告日

today = dt.datetime.today()
date_3yrs_ago = today.replace(year=today.year - 3)
report_dates_begin = nearest_report_date(date_3yrs_ago.strftime("%Y%m%d"))
current_date = today.strftime("%Y%m%d")

In [58]:
# 获取所有目标债券型基金（清算结束日期晚于当前日）

security_ids = read_sql(f"""
select SECURITYID from TQ_FD_TYPECLASS 
WHERE
    ISVALID = 1 AND
    L1CODE = 3 AND
    (ENDDATE>='{report_dates_begin}' or ENDDATE = '19000101') 
ORDER BY SECURITYID;
""")["SECURITYID"].unique().tolist()
sec_id_strs = ",".join(["'" + s + "'" for s in security_ids])

not_liq_sec_ids = fd_basicinfo(security_ids, current_date)["SECURITYID"].unique().tolist()
sec_id_strs = ",".join(["'" + s + "'" for s in not_liq_sec_ids])

In [59]:
# 获取相关组合情况

query = f"""
SELECT SECURITYID, REPORTDATE, BDRTO, CONVBDRTO, EQUITYINVERTO from TQ_FD_ASSETPORTFOLIO
WHERE
    REPORTDATE >= '{report_dates_begin}' AND
    ISVALID = 1 AND
    SECURITYID in ({sec_id_strs}) ORDER BY SECURITYID, REPORTDATE
"""
portfolios = read_sql(query).fillna(0.0)
portfolios["CONVBDRTO_TO_BDRTO"] = portfolios["CONVBDRTO"] / portfolios["BDRTO"] * 100

In [60]:
last_portfolio = portfolios.groupby("SECURITYID").last()[["BDRTO"]]
last_4_cov_portfolio = portfolios.groupby("SECURITYID").rolling(window=4).mean().groupby(level=0).last()[["CONVBDRTO_TO_BDRTO"]]
last_4_stk_portfolio = portfolios.groupby("SECURITYID").rolling(window=4).mean().groupby(level=0).last()[["EQUITYINVERTO"]]

In [61]:
agg_port = pd.concat([last_portfolio, last_4_cov_portfolio, last_4_stk_portfolio], axis=1).reset_index()
agg_port = pd.merge(agg_port, fd_typeclass(agg_port.SECURITYID, current_date))

In [62]:
agg_port

,SECURITYID,BDRTO,CONVBDRTO_TO_BDRTO,EQUITYINVERTO,L1CODE,L1NAME,L2CODE,L2NAME
0,1030000011,85.80,23.140246,2.7075,3,债券基金,3.2,普通债券型基金
1,1030000012,85.80,23.140246,2.7075,3,债券基金,3.2,普通债券型基金
2,1030000014,80.75,2.309944,10.3475,3,债券基金,3.2,普通债券型基金
3,1030000015,80.75,2.309944,10.3475,3,债券基金,3.2,普通债券型基金
4,1030000022,113.76,16.596532,0.0000,3,债券基金,3.2,普通债券型基金
...,...,...,...,...,...,...,...,...
1284,1030010008,0.00,NaN,0.0000,3,债券基金,3.6,短期理财债券型基金
1285,1030013271,114.12,0.020749,0.0000,3,债券基金,3.2,普通债券型基金
1286,1030013280,93.34,13.607275,15.0875,3,债券基金,3.2,普通债券型基金
1287,1030013281,93.34,13.607275,15.0875,3,债券基金,3.2,普通债券型基金


In [44]:
# 分类
agg_port.loc[(agg_port["BDRTO"] >= 80) & (agg_port["CONVBDRTO_TO_BDRTO"] >= 80), "债券型（子类）"] = "可转债基金"
agg_port.loc[(agg_port["BDRTO"] >= 80) & (agg_port["CONVBDRTO_TO_BDRTO"] >= 5) & (agg_port["CONVBDRTO_TO_BDRTO"] < 80) & (agg_port["EQUITYINVERTO"] <= 0.1), "债券型（子类）"] = "可投转债型债券基金"
agg_port.loc[(agg_port["BDRTO"] >= 80) & (agg_port["EQUITYINVERTO"] >= 0.1) & (~agg_port["债券型（子类）"].isin(["可投转债型债券基金", "可转债基金"])), "债券型（子类）"] = "可投股票型债券基金"
agg_port.loc[(agg_port["BDRTO"] >= 80) & (agg_port["L2NAME"] == "指数债券型基金") & (~agg_port["债券型（子类）"].isin(["可投转债型债券基金", "可转债基金"])), "债券型（子类）"] = "被动指数型债券基金"

In [45]:
agg_port.groupby("债券型（子类）").count()

,SECURITYID,BDRTO,CONVBDRTO_TO_BDRTO,EQUITYINVERTO,L1CODE,L1NAME,L2CODE,L2NAME
债券型（子类）,,,,,,,,
可投股票型债券基金,374,374,374,374,374,374,374,374
可投转债型债券基金,118,118,118,118,118,118,118,118
可转债基金,52,52,52,52,52,52,52,52
被动指数型债券基金,17,17,17,17,17,17,17,17


In [46]:
agg_port.groupby("债券型（子类）").count()

,SECURITYID,BDRTO,CONVBDRTO_TO_BDRTO,EQUITYINVERTO,L1CODE,L1NAME,L2CODE,L2NAME
债券型（子类）,,,,,,,,
可投股票型债券基金,374,374,374,374,374,374,374,374
可投转债型债券基金,118,118,118,118,118,118,118,118
可转债基金,52,52,52,52,52,52,52,52
被动指数型债券基金,17,17,17,17,17,17,17,17


In [47]:
agg_port = pd.merge(agg_port, fd_basicinfo(agg_port.SECURITYID, current_date))

In [50]:
agg_port[agg_port["债券型（子类）"] == "被动指数型债券基金"].sort_values("FSYMBOL")

,SECURITYID,BDRTO,CONVBDRTO_TO_BDRTO,EQUITYINVERTO,L1CODE,L1NAME,L2CODE,L2NAME,债券型（子类）,FDNAME,SNAMECOMP,FSYMBOL
183,1030001112,94.67,0.000000e+00,0.000,3,债券基金,3.4,指数债券型基金,被动指数型债券基金,亚债中国债券指数基金,华夏亚债中国债券指数,001021
184,1030001113,94.67,0.000000e+00,0.000,3,债券基金,3.4,指数债券型基金,被动指数型债券基金,亚债中国债券指数基金,华夏亚债中国债券指数,001023
654,1030005384,103.71,0.000000e+00,0.000,3,债券基金,3.4,指数债券型基金,被动指数型债券基金,易方达中债3-5年期国债指数证券投资基金,None,001512
1136,1030008768,121.80,0.000000e+00,0.000,3,债券基金,3.4,指数债券型基金,被动指数型债券基金,中融上海清算所银行间1-3年中高等级信用债指数发起式证券投资基金,中融银行间1-3年中高等级信用债指数,003081
1137,1030008769,121.80,0.000000e+00,0.000,3,债券基金,3.4,指数债券型基金,被动指数型债券基金,中融上海清算所银行间1-3年中高等级信用债指数发起式证券投资基金,中融银行间1-3年中高等级信用债指数,003082
1068,1030008391,84.71,0.000000e+00,0.000,3,债券基金,3.4,指数债券型基金,被动指数型债券基金,中融上海清算所银行间1-3年高等级信用债指数发起式证券投资基金,中融银行间1-3年高等级信用债指数,003083
1069,1030008392,84.71,0.000000e+00,0.000,3,债券基金,3.4,指数债券型基金,被动指数型债券基金,中融上海清算所银行间1-3年高等级信用债指数发起式证券投资基金,中融银行间1-3年高等级信用债指数,003084
931,1030007676,106.49,0.000000e+00,0.000,3,债券基金,3.4,指数债券型基金,被动指数型债券基金,易方达中债7-10年期国开行债券指数证券投资基金,易方达中债7-10年期国开行债券指数,003358
929,1030007664,98.43,0.000000e+00,0.000,3,债券基金,3.4,指数债券型基金,被动指数型债券基金,广发中债7-10年期国开行债券指数证券投资基金,广发中债7-10年国开债指数,003376
930,1030007665,98.43,0.000000e+00,0.000,3,债券基金,3.4,指数债券型基金,被动指数型债券基金,广发中债7-10年期国开行债券指数证券投资基金,广发中债7-10年国开债指数,003377


In [52]:
agg_port[agg_port["FSYMBOL"] == "002659"]

,SECURITYID,BDRTO,CONVBDRTO_TO_BDRTO,EQUITYINVERTO,L1CODE,L1NAME,L2CODE,L2NAME,债券型（子类）,FDNAME,SNAMECOMP,FSYMBOL
